In [1]:
!pip install --upgrade boto3 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install --upgrade sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/85/aa/7f8313a310325d9c1ef0b8c34295018637ed4989bdef13c2831758561780/boto3-1.28.44-py3-none-any.whl (135 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f0/e8/821ce5cec175739d83d8c1f36f347e9615c19aa133c6038804c7e1d61fa5/botocore-1.31.44-py3-none-any.whl (11.2 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.28
    Uninstalling botocore-1.31.28:
      Successfully uninstalled botocore-1.31.28
  Attempting uninstall: boto3
    Found existing installation: boto3 1.28.28
    Uninstalling boto3-1.28.28:
      Successfully uninstalled boto3-1.28.28
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.28 requires botocore==1.31.28, but you have botocore 1.31.44 which is incompatible.
Looking in ind

In [2]:
# For notebook instances (Amazon Linux)
!sudo yum update -y
!sudo yum install amazon-linux-extras
!sudo amazon-linux-extras install epel -y
!sudo yum update -y
!sudo yum install git-lfs git -y

Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
amzn2extra-docker                                        | 3.0 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
amzn2extra-python3.8                                     | 3.0 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
libnvidia-container/x86_64/signature                     |  833 B     00:00     
libnvidia-container/x86_64/signature                     | 2.1 kB     00:00 !!! 
neuron                                                   | 2.9 kB     00:00     
neuron/prima

In [1]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws-cn:iam::145525377082:role/NotebookStack-SmartSearchNotebookRole6F6BB12B-1B2P6EC8MOL1B
sagemaker bucket: sagemaker-cn-north-1-145525377082
sagemaker 

In [2]:
!mkdir code

In [3]:
%%writefile code/inference.py


# from transformers import AutoTokenizer, AutoModel
import torch

import json
# import torch.nn.functional as F

from transformers import BertTokenizer, BertModel

# Helper: Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def model_fn(model_dir):
    print("=================model_fn_Start=================")
    tokenizer = BertTokenizer.from_pretrained(model_dir)
    model = BertModel.from_pretrained(model_dir)
    print("=================model_fn_End=================")
    return model, tokenizer

def input_fn(request_body, request_content_type):
    """
    Deserialize and prepare the prediction input
    """
    print(f"=================input_fn=================\n{request_content_type}\n{request_body}")
    input_data = json.loads(request_body)
    print(input_data)
    return input_data

def predict_fn(data, model_and_tokenizer):
    
    print(f"=================predict_fn=================\n")
    print(data)
    # destruct model and tokenizer
    model, tokenizer = model_and_tokenizer

    # Tokenize sentences
    sentences = data.pop("inputs", data)
    
    ebs = []
    
    for sentence in sentences:
    
        encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

        with torch.no_grad():
            model_output = model(**encoded_input)
        # Perform pooling. In this case, mean pooling.
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        
        ebs.append(sentence_embeddings.tolist())

    # return dictonary, which will be json serializable
    return ebs
    # return sentence_embeddings.tolist()
    # return {"vectors": sentence_embeddings.tolist()}

def output_fn(prediction, content_type):
    """
    Serialize and prepare the prediction output
    """
    print(f"=================out_fn_type=================\n")
    print(content_type)
    return [prediction]


Writing code/inference.py


In [11]:
repository = "lilbay3000/text2vec-base-chinese"
url_suffix = repository + ".git"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sess.default_bucket()}/custom_inference/{model_id}/model.tar.gz"

In [12]:
!git lfs install
!git clone https://www.wisemodel.cn/$url_suffix

Error: failed to call git rev-parse --git-dir: exit status 128 : fatal: detected dubious ownership in repository at '/home/ec2-user/SageMaker/guidance-for-custom-search-of-an-enterprise-knowledge-base-on-aws'
To add an exception for this directory, call:

	git config --global --add safe.directory /home/ec2-user/SageMaker/guidance-for-custom-search-of-an-enterprise-knowledge-base-on-aws

Git LFS initialized.
Cloning into 'text2vec-base-chinese'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (23/23), 163.31 KiB | 16.33 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Filtering content: 100% (3/3), 1.46 GiB | 23.05 MiB/s, done.


In [13]:
!ls
!cp -r code/ $model_id/code/

code  shibing624-model-deploy-pip-cn.ipynb  text2vec-base-chinese


In [14]:
%cd $model_id
!tar zcvf model.tar.gz *

/home/ec2-user/SageMaker/guidance-for-custom-search-of-an-enterprise-knowledge-base-on-aws/Embedding_Model/shibing624-model-deploy-pip-cn/text2vec-base-chinese
1_Pooling/
1_Pooling/config.json
code/
code/inference.py
config.json
logs.txt
model.tar.gz
modules.json
onnx/
onnx/model.onnx
onnx/config.json
onnx/tokenizer_config.json
onnx/special_tokens_map.json
onnx/tokenizer.json
onnx/vocab.txt
pytorch_model.bin
README.md
sentence_bert_config.json
special_tokens_map.json
tokenizer_config.json
vocab.txt


In [15]:
!aws s3 cp model.tar.gz $s3_location

upload: ./model.tar.gz to s3://sagemaker-cn-north-1-145525377082/custom_inference/text2vec-base-chinese/model.tar.gz


In [16]:
import sagemaker
from sagemaker.huggingface.model import HuggingFaceModel

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_location,       # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.12",  # transformers version used
   pytorch_version="1.9",        # pytorch version used
   py_version='py38'            # python version used
)

endpoint_name = 'huggingface-inference-eb'
# instance_type = 'ml.g4dn.2xlarge'
instance_type = 'ml.g4dn.xlarge'
instance_count = 1

# deploy the endpoint endpoint
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = huggingface_model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
-------!

In [20]:
# Inference testing
import time
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

hfp = sagemaker.huggingface.model.HuggingFacePredictor('huggingface-inference-eb')

embedding = hfp.predict({'inputs':['你好','你好吗']})[0][0][0]
print(embedding)
print(len(embedding))

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
[-0.21393102407455444, 0.3746872842311859, 0.6576342582702637, 0.06952203065156937, 0.36052781343460083, -0.510812520980835, 0.3674842119216919, -0.32344022393226624, -0.6663990616798401, 0.7574434280395508, 0.5047809481620789, 0.48273366689682007, 0.04197601228952408, -0.38980618119239807, -0.13082703948020935, 0.14291933178901672, -0.13354630768299103, -0.3493451476097107, -0.35633230209350586, -0.928504467010498, -0.5945854783058167, 0.7669195532798767, 0.04896663874387741, 0.22816094756126404, -0.30049240589141846, 1.0198930501937866, -0.08743584156036377, 0.44177818298339844, 1.431441068649292, 0.6234298348426819, -0.17712412774562836, 0.7905667424201965, -0.26568591594696045, 0.008183373138308525, -0.9489567279815674, -0.31399136781692505, -0.12018207460641861, 1.00497162342071